In [1]:
# Import python modules
import os
import datetime
import pandas as pd
import numpy as np

In [2]:
# Requires the following databses
# phenotypes_db.txt : database of genes and known phenotypes associations
# mim2gene_mod.txt : gene mim ID
# pLI.gnomad.txt : gnomad pLI calculation per gene
localfiles = "/Users/boris/GoogleDrive/UDD/research/bioinformatics/SABIO/projects/01_DECIPHERD/00_run_pipeline/"
phenotypes_db = localfiles + "/databases/phenotypes_db.txt"
mim2gene_db = localfiles + "/databases/mim2gene_mod.txt"
#pLI_db = localfiles + "/databases/pLI.gnomad.txt"

In [3]:
# Define input/ouput 
timestamp = str(datetime.datetime.now().strftime("%Y-%m-%d_%H_%M"))

decipherd = "/Users/boris/GoogleDrive/UDD/research/bioinformatics/SABIO/projects/01_DECIPHERD/"
baylorva = "baylor/LABWORK/exome_analysis/"

famid = "BH13053"
proband = famid + "-1" + "_filtered_reannotated.csv"
outband = famid + "-1" + "_decipherd.xlsx"

infile = decipherd+baylorva+proband
outfile = decipherd+baylorva+timestamp+"_"+outband

# Define comparison file
parent = decipherd + baylorva + famid + "-2" + "_filtered_reannotated.csv"

In [4]:
# The function takes frequency values. If frequency values exist, they are compared to the AF<=0.01, otherwise, if
# no info is available, keep the site anyway
def freq_check(values, NotANum):
    outvals = [str(v) for v in values]
    results = 0
    for freq in values:
        if freq is NotANum:
            freq2 = 0
        else:
            freq2 = freq
        try:
            f = float(freq2)
            if f <= 0.015:
                results+-1
            else:
                results+=1
        except:
            results+=0
    if results <= 0:
        return pd.Series(["rare|unknown", "|".join(outvals)])
    else:
        return pd.Series(["common", "|".join(outvals)])

# The function takes impact values. If impact values exist, at least two criteria agree with functional impact, otherwise, if
# no info is available, keep the site anyway
def impact_check(values, NotANum):
    outvals = [str(v) for v in values]
    results = 0
    for impact in values:
        if impact is NotANum:
            imp2 = "D"
        else:
            imp2 = impact
        try:
            i = str(imp2)
            if i in ["D", "P", "H", "M", "A"]:
                results+=-1
            elif i != '.':
                results+=1
            else:
                results+=0
        except:
            results+=0
    if results <= 0:
        return pd.Series(["affecting|unknown", "|".join(outvals)])
    else:
        return pd.Series(["unaffecting", "|".join(outvals)])

# This function check for DP of the variant call >=50
def check_DP(depth, cutoff=50):
    #dp = depth.strip().split(":")[2]
    dp = depth
    if float(dp) >= cutoff:
        return True
    else:
        return False

# This function check for GT 
def check_GT(genotype, gt_default='1/1'):
    gt = genotype.strip().split(":")[0]
    if gt == gt_default:
        return True
    else:
        return False

# This function splits DP and GT 
def split_format(fmt, n):
    out = fmt.strip().split(":")[n]
    if n in [2,3]:
        return int(out)
    else:
        return out

# Determine zygosity
def zygosity(genotype):
    A1,A2 = genotype.split("/")
    if A1==A2:
        return "hom"
    else:
        return "het"

def compound(genes, gene_occ):
    gene_list = str(genes).split(',')
    occurence = [gene_occ[gene] for gene in gene_list]
    if len([i for i in occurence if i>1]) !=0:
        return 'true'
    else:
        return 'false'

def pheno_dic(entry):
    genes = entry[0]
    pheno = entry[1]
    for gene in genes.split(','):
        return (gene,pheno)

def get_pheno(genes,phenodb):
    phenotypes = []
    for gene in str(genes).split(','):
        try:
            phenotypes.append(phenodb[gene])
        except:
            pass
    return ''.join(list(set(phenotypes)))

def get_info(info, val):
    info_values = info.split(';')
    tmp = [i for i in info_values if i.startswith(val)]
    try:
        out = float(tmp[0].split("=")[1])
    except:
        out = ''
    return out

def check_gene(genes, db):
    genelist =  str(genes).split(',')
    match = [g for g in genelist if g in db.keys()]
    if match:
        n, feat, hpo = pheno_db[match[0]]
        return pd.Series(["yes", n, feat, hpo])
    else:
        return pd.Series(["no",0,'-','-'])

In [5]:
# Read genomic data table generated by uploading HC-GATK raw VCF to http://wannovar.wglab.org/ 
data = pd.read_csv(infile, low_memory=False)
parent = pd.read_csv(parent, low_memory=False)

In [6]:
# Add MIM phenotypes info
pheno_0 = pd.read_csv(phenotypes_db, header=None, sep='\t')
pheno_0.columns = ['Raw_gene_name','Phenotypes']
pheno_1 = pheno_0.dropna()
phenodbase = dict(list(pheno_1.apply(pheno_dic, axis=1)))
data["Phenotypes"] = data['Raw_gene_name'].apply(get_pheno, phenodb=phenodbase)

del pheno_0
del pheno_1
del phenotypes_db

In [7]:
#Add MIM gene info
omim = pd.read_csv(mim2gene_db, low_memory=False, header=None, sep='\t')
omim.columns = ['mim', 'Raw_gene_name']
weblinks = omim.mim.apply(lambda x: '=HYPERLINK("https://www.omim.org/entry/%s", "%s")' % (str(x), str(x)))
omim['mim'] = weblinks
data_2 = pd.merge(data, omim, how = 'left')

del omim
del mim2gene_db

In [8]:
del data
data = data_2

In [9]:
Snp = []
for snp, chrom, pos, ref,alt in data[['DBSNP_rsid', 'CHROM','POS','REF','ALT']].values:
#    if str(snp) != 'nan':
#        Snp.append('=HYPERLINK("https://www.ncbi.nlm.nih.gov/snp/%s", "%s")' % (snp, snp))
#    else:
        loc = "%s:%s%s>%s" % (chrom,pos,ref,alt)
        Snp.append('=HYPERLINK("https://gnomad.broadinstitute.org/region/%s-%s-%s", "%s")' % (chrom,pos,pos,loc))

data['Gnomad'] = Snp       

In [10]:
marrvel = []
for chrom, pos, ref, alt in data[['CHROM','POS','REF','ALT']].values:
    loc = "%s:%s%%20%s>%s" % (chrom,pos,ref,alt)
    marrvel.append('=HYPERLINK("http://marrvel.org/search/variant/%s", "%s")' % (loc,loc))
data['marrvel'] = marrvel 

In [11]:
# Extract frequency databases column names
freq_cols_tmp = [col for col in list(data.columns) if col.endswith("_ALL") or col.endswith("_all")]
freq_cols = [freq_cols_tmp[i] for i in [0,2,3,4]]

# Extract Clinvar column
#clinVar_cols = [col for col in list(data.columns) if col.startswith("ClinVar")][1:]
clinVar_cols = ["CLNSIG"]

# Define impact criterias to be considered
impact_cols = ["SIFT_pred", "Polyphen2_HDIV_pred", "Polyphen2_HVAR_pred", 
               "MutationTaster_pred", "MutationAssessor_pred"]

In [12]:
allele_freq = data[freq_cols].apply(freq_check, axis=1, NotANum=np.nan)
allele_freq.columns = ["db_AF_class", "db_AF_class_values"]

In [13]:
data_tmp = pd.concat([data,allele_freq], axis=1)
data = data_tmp
del data_tmp

In [14]:
pred_effect = data[impact_cols].apply(impact_check, axis=1, NotANum=np.nan)
pred_effect.columns = ["pred_effect_class", "pred_effect_class_values"]

In [15]:
data_tmp = pd.concat([data,pred_effect], axis=1)
data = data_tmp
del data_tmp

In [16]:
data['Parent'] = list(data['key'].str.replace("-1_", "-2_").isin(parent['key']))

In [17]:
getcols = ["Zygosity", "Compound_het", "Potential_Denovo", "Potential_InTrans", 
           "Potential_InCis", "Potential_Parent1", "Potential_Parent2",
          "Raw_gene_name", "Mutation_type_.Refseq.", "Mutation_type_.UCSC.", "pLI", "CADD_phred"]

uddcols = list(data.columns)[175:]
bcmcols = [col for col in list(data.columns)[:175] if col not in getcols]

#outcols = ["BR_filter"] + ["Parent"] + uddcols + getcols + bcmcols 
outcols = uddcols + ['ARO'] + getcols + bcmcols 

In [18]:
data.drop_duplicates(inplace=True)
data.CHROM.replace(['X','Y'],[23,24], inplace=True)
tmp = pd.to_numeric(data.CHROM)
data.drop('CHROM',axis=1)
data['CHROM'] = tmp

In [19]:
tmp = pd.to_numeric(data.pLI.replace('nan', '-1'))
data['pLI'] = tmp
del tmp

In [20]:
tmp = pd.to_numeric(data.CADD_phred.replace('.', '-1'))
data['CADD_phred'] = tmp
del tmp

In [23]:
data.sort_values(by=['CHROM','POS'], ascending=True, inplace=True)

In [24]:
data.drop_duplicates(inplace=True)
data['ARO'] = data.Phenotypes.str.contains('recessive') & ~(data.Phenotypes.str.contains('dominant'))

In [25]:
writer = pd.ExcelWriter(outfile)
data[outcols].to_excel(writer, famid+"-1", index=False)
writer.save()